## Firmas digitales

Las firmas digitales permiten a través de la criptografía, verificar que un documento o un mensaje no ha sido alterado durante su transmisión. También permiten confirmar la identidad de la persona o entidad que crea un documento digital.

En esta actividad crearemos documentos y los firmaremos digitalmente para poner en práctica la fundamentación teórica adquirida en el curso de ciberseguridad

In [1]:
import os
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet
import base64
import json
from datetime import datetime

### Claves simétricas para la identidad
Para poder firmar digitalmente documentos, necesitamos crear un par de claves (privada y pública).
Recuerda siempre que la clave privada debe estar protegida y no se debe compartir en ningún caso. si esta clave se filtrase
otra persona se puede hacer pasar por nosotros y no habría forma de detectar digitalmente que no somos el remitente

In [3]:
# utilizaremos las mismas funciones del laboratorio de claves asimétricas

def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )


In [4]:
# generando unas claves para la actividad:

generar_claves_asimetricas()


Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.


### Documentos y firmas digitales

Con la clave privada y pública podemos realizar la firma digital del documento. Como ejercicio nuestros documentos serán archivos en formato JSON que contendrán ciertos datos. Cada estudiante debe utilizar la función de  ´´´crear_documento´´´ para generar un documento propio y luego firmarlo.

La firma digital funciona de forma similar a cuando se cifran mensajes con claves asimétricas, en este caso el proceso consta de dos pasos. El primero, calcular el resultado de la función hash de un documento y el segundo, cifrar el hash con una clave privada para validar la identidad del emisor del documento y el contenido del documento.

Cuando firmarmos el documento, lo que hacemos es cargar el JSON como un string en memoria y luego lo codificamos como bits. A estos bits se les calcula una huella digital con una función de hash (sha256 en este caso). La huella es una cadena de longitud fija que representa de manera única el contenido del documento. Si algún bit cambia, la firma cambia totalmente.

Luego, en la función de firmar documento, la firma se codifica con la clave privada, con esto garantizamos que solo quien posee la clave (el remitente real) es el emisor del documento.

Finalmente, la función de firmar documento guarda la firma, que es, el hash del documento cifrado con la clave privada.




In [26]:
def crear_documento(autor, titulo, contenido, nombre_archivo="documento.json"):
    """
    Crea un documento en formato JSON para ser firmado.

    Args:
        autor (str): Nombre del autor del documento
        titulo (str): Título del documento
        contenido (str): Contenido del documento
        nombre_archivo (str): Nombre del archivo donde se guardará el documento

    Returns:
        dict: El documento creado
    """
    documento = {
        "autor": autor,
        "titulo": titulo,
        "contenido": contenido,
        "fecha_creacion": datetime.now().isoformat()
    }

    with open(nombre_archivo, "w") as f:
        json.dump(documento, f, indent=4)

    print(f"Documento creado y guardado en: {nombre_archivo}")
    return documento


In [27]:
crear_documento("Autor de ejemplo (aca va tu nombre)",
                "Declaracion firmada"
                "En este documento oficial listamos las vulnerabilidades de seguridad encontradas en nuestros sistemas"
                "no debe compartirse y debe verificarse su origen antes de proceder con los pasos:"
                "paso 1: realizar revisiones"
                "paso 2: monitorear sistemas"
                "paso 3: Actualizar componentes"
                "paso 4: Revisar registros",
                "declaracion.json"
                )


#NOTA:  revisar que el documento exista en el disco antes de proceder con la actividad!


Documento creado y guardado en: documento.json


{'autor': 'Autor de ejemplo (aca va tu nombre)',
 'titulo': 'Declaracion firmadaEn este documento oficial listamos las vulnerabilidades de seguridad encontradas en nuestros sistemasno debe compartirse y debe verificarse su origen antes de proceder con los pasos:paso 1: realizar revisionespaso 2: monitorear sistemaspaso 3: Actualizar componentespaso 4: Revisar registros',
 'contenido': 'declaracion.json',
 'fecha_creacion': '2025-04-10T11:38:17.979188'}

In [37]:
# función para firmar digitalmente un documento
def firmar_documento(ruta_documento, ruta_firma="firma_digital.bin"):
    """
    Firma digitalmente un ruta_documento utilizando una clave privada RSA.

    Args:
        ruta_clave_privada (str): Ruta al archivo de clave privada PEM
        documento (dict): Documento a firmar
        ruta_firma (str): Ruta donde se guardará la firma

    Returns:
        bytes: La firma digital
    """
    # Cargamos la clave privada
    private_key = cargar_clave_privada()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()

    # Firmamos el mensaje
    firma = private_key.sign(
        mensaje,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    # Guardamos la firma en un archivo
    with open(ruta_firma, "wb") as f:
        f.write(firma)

    print(f"Documento firmado. Firma guardada en: {ruta_firma}")

    # También guardamos la firma en formato base64 para visualización
    firma_base64 = base64.b64encode(firma).decode('utf-8')
    with open(f"{ruta_firma}.b64", "w") as f:
        f.write(firma_base64)

    print(f"Firma en formato base64 guardada en: {ruta_firma}.b64")

    return firma


In [39]:
firma = firmar_documento("documento.json")

Documento firmado. Firma guardada en: firma_digital.bin
Firma en formato base64 guardada en: firma_digital.bin.b64


### Verificación de las firmas digitales

Para verificar que un documento es auténtico y su contenido es el mismo con el que el emisor lo publicó, necesitamos contar con tres cosas que son:
    - El documento a validar
    - La clave pública del emisor del documento
    - el archivo con la firma digital

La verificación recalcula el hash del documento recibido, usa la clave pública para descifrar la firma digital y compara el hash calculado con el recibido (firma). Si ambos coinciden, se puede entender que el documento no ha sido alterado y que la persona que lo generó es quien dice ser.

In [44]:
def verificar_firma(ruta_documento, ruta_firma):
    """
    Verifica la firma digital de un documento utilizando una clave pública RSA.

    Args:
        ruta_clave_publica (str): Ruta al archivo de clave pública PEM
        documento (dict): Documento cuya firma se va a verificar
        ruta_firma (str): Ruta al archivo de firma

    Returns:
        bool: True si la firma es válida, False en caso contrario
    """
    # Cargamos la clave pública
    public_key = cargar_clave_publica()

    # Cargamos la firma
    with open(ruta_firma, "rb") as f:
        firma = f.read()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()


    try:
        # Verificamos la firma
        public_key.verify(
            firma,
            mensaje,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("¡La firma es válida! El documento es auténtico y no ha sido modificado.")
        return True
    except Exception as e:
        print("¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.")
        print(f"Error: {e}")
        return False


In [45]:
verificacion = verificar_firma("documento.json", "firma_digital.bin")



¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.
Error: 


### Ejercicios en clase:

1. Intercambie permita que los estudiantes en grupos de hasta 5 personas, creen una carpeta con sus documentos, claves públicas (con el nombre de cada uno) y firmas. Con esta información cada estudiante debe identificar qué documento pertenece a cada emisor.
2. Pida a los estudiantes que alteren una letra de los documentos o una parte del contenido y recalculen la firma. ¿Es valido el documento alterado? Concluya con ellos sobre la seguridad del uso de firmas digitales.
3. Pida a los estudiantes que, en lugar de crear documentos en formato json, creen documentos en formato txt y word. Ahora modifiquen el código para firmar digitalmente alguno de estos documentos y repitan el ejercicio 1 pero en parejas. Discuta con ellos si la implementación de firmas digitales es suficiente para proteger la autenticidad de los documentos.
4. Debata con los estudiantes qué se debería hacer en caso de que una clave privada se filtre.
5. Con las funciones creadas, diseñe con los estudiantes un sistema de cadenas de firmas, en la que una persona genera y firma un documento. Una segunda persona altera y firma el documento y una tercera persona también altera el documento y lo firma. ¿Cómo se puede validar la identidad de cada firmante?
